In [ ]:
import os
import sys
sys.path.append('/Automatic-Circuit-Discovery/')
sys.path.append('..')
import IPython
ipython = get_ipython()
if ipython is not None:
    ipython.magic('load_ext autoreload')
    ipython.magic('autoreload 2')
import torch as t
from torch import Tensor
import numpy as np
import matplotlib.pyplot as plt
import json

from acdc.TLACDCExperiment import TLACDCExperiment
from acdc.docstring.utils import get_all_docstring_things
from utils.prune_utils import get_3_caches, split_layers_and_heads
from ACDCPPExperiment import ACDCPPExperiment
device = t.device("cuda" if t.cuda.is_available() else "cpu")
print(device)

In [ ]:
all_docstring_items = get_all_docstring_things(num_examples=40, seq_len=5, device=device, metric_name='docstring_metric', correct_incorrect_wandb=False)

tl_model = all_docstring_items.tl_model
validation_metric = all_docstring_items.validation_metric
validation_data = all_docstring_items.validation_data
validation_labels = all_docstring_items.validation_labels
validation_patch_data = all_docstring_items.validation_patch_data
test_metrics = all_docstring_items.test_metrics
test_data = all_docstring_items.test_data
test_labels = all_docstring_items.test_labels
test_patch_data = all_docstring_items.test_patch_data

In [ ]:
def abs_docstring_metric(logits):
    return -abs(test_metrics['docstring_metric'](logits))

# ACDC++ run

In [ ]:
RUN_NAME = "docstring_absval_edge"
# acdcpp_docstring_convergence_thresh = 1#0.03
# acdc_paper_thresh = 1#0.067
# num_thresholds = 2#5

ACDCPP_THRESHOLDS = 7 / np.linspace(2, 1500, 20)
ACDC_THRESHOLDS = ACDCPP_THRESHOLDS

print(f"{ACDCPP_THRESHOLDS=}")
print(f"{ACDC_THRESHOLDS=}")

In [ ]:
combined_exp = ACDCPPExperiment(
    tl_model,
    test_data,
    test_patch_data,
    acdc_metric=test_metrics['docstring_metric'],
    acdcpp_metric=abs_docstring_metric,
    acdc_thresholds=ACDC_THRESHOLDS,
    acdcpp_thresholds=ACDCPP_THRESHOLDS,
    run_name=RUN_NAME,
    verbose=False,
    attr_absolute_val=True,
    save_graphs_after=0,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
    thresholds_combined=False,
)
present_edge_attrs, num_passes, acdcpp_attrs = combined_exp.run()